In [ ]:
import os
project_dir = "/Volumes/mnt/Workspace/ai-trading-agents"
os.chdir(project_dir)

In [ ]:
get_ipython().run_line_magic('reload_ext', 'autoreload')  # type: ignore
get_ipython().run_line_magic('autoreload', '2')

In [ ]:
from src import secrets
from src import db

In [ ]:
SECRETS = secrets.load()

ENV = os.environ.get("ENV", "dev")

In [ ]:
await db.connect()

In [ ]:
run_id = "e738e2a0-8d02-4d7a-af3b-a7ac81212a01"

In [ ]:
run = await db.prisma.run.find_unique(where={"id": run_id})

In [ ]:
run

In [ ]:
if not run:
    logger.error(f"Run {run_id} not found")
    exit(2)

if run.status == "SUCCESS":
    logger.error(f"Run {run_id} is finished")
    exit(2)

In [ ]:
start_messages = [HumanMessage(content=DEFAULT_USER_PROMPT)]
deserialized_messages = await restore_messages(run_id)

In [ ]:
if deserialized_messages == "ERROR":
    logger.error(f"Failed to restore messages for run {run_id}")
    exit(2)

In [ ]:
if deserialized_messages:
    start_messages = deserialized_messages

In [ ]:
await db.prisma.run.update(where={"id": run_id}, data={"status": RunStatus.RUNNING})

In [ ]:
context = await build_context(run=run)

In [ ]:
if context == "ERROR":
    logger.error(f"Failed to build context for run {run.id}")
    exit(2)

In [ ]:
agent_graph = await build_chief_investment_officer_agent(context)

In [ ]:
# response = await agent_graph.ainvoke(
#     {"messages": start_messages},
#     context=context,
#     stream_mode="values",
#     print_mode="updates",
# )

In [ ]:
events = agent_graph.astream(
    input={
        "messages": start_messages,  # type: ignore
    },
    context=context,
    stream_mode="values",
)
async for event in events:
    if "messages" in event:
        message = event["messages"][-1]
        message.pretty_print()

In [ ]:
await db.prisma.run.update(where={"id": run_id}, data={"status": RunStatus.SUCCESS})